<a href="https://colab.research.google.com/github/aggelospsiris/NB-KNN-SMV/blob/main/dim_reduction_with_t_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from math import sqrt
import warnings
warnings.filterwarnings('ignore')
from google.colab import drive 
drive.mount('/drive')
df=pd.read_csv('/drive/My Drive/Colab Notebooks/decision_theory/processed_data.csv')

Mounted at /drive


In [ ]:
df = df.drop(columns=['Unnamed: 0'])

In [ ]:
from scipy import stats

tests = pd.DataFrame(columns=['name','t_statistics','p_value'])

#compute students t_test scores 
gender_test = stats.ttest_ind(df['gender'], df['healthy'])
tests.loc[len(tests.index)] = ['gender', gender_test.statistic, gender_test.pvalue] 
age_test = stats.ttest_ind(df['age'], df['healthy'])
tests.loc[len(tests.index)] = ['age', age_test.statistic, age_test.pvalue] 
tb_test = stats.ttest_ind(df['tb'], df['healthy'])
tests.loc[len(tests.index)] = ['tb', tb_test.statistic, tb_test.pvalue] 
db_test = stats.ttest_ind(df['db'], df['healthy'])
tests.loc[len(tests.index)] = ['db', db_test.statistic, db_test.pvalue] 
tp_test = stats.ttest_ind(df['tp'], df['healthy'])
tests.loc[len(tests.index)] = ['tp', tp_test.statistic, tp_test.pvalue] 
albumin_test = stats.ttest_ind(df['albumin'], df['healthy'])
tests.loc[len(tests.index)] = ['albumin', albumin_test.statistic, albumin_test.pvalue] 
A_G_test = stats.ttest_ind(df['A/G ratio'], df['healthy'])
tests.loc[len(tests.index)] = ['A/G', A_G_test.statistic, A_G_test.pvalue] 
SGPT_test = stats.ttest_ind(df['SGPT'], df['healthy'])
tests.loc[len(tests.index)] = ['SGPT', SGPT_test.statistic, SGPT_test.pvalue] 
SGOT_test = stats.ttest_ind(df['SGOT'], df['healthy'])
tests.loc[len(tests.index)] = ['SGOT', SGOT_test.statistic, SGOT_test.pvalue] 
Alkphos_test = stats.ttest_ind(df['Alkphos'], df['healthy'])
tests.loc[len(tests.index)] = ['Alkphos', Alkphos_test.statistic, Alkphos_test.pvalue] 
#keep only the 5 with the smallest p_values cause High p-values indicate that your 
#evidence is not strong enough to suggest an effect exists in the population. 
#An effect might exist but it's possible that the effect size is too small,
# the sample size is too small, or there is too much variability for the 
#hypothesis test to detect it
tests['p_value'].nsmallest(5)

1     0.000000e+00
7     0.000000e+00
8    7.698876e-285
4    8.388832e-138
0     4.717427e-80
Name: p_value, dtype: float64

In [ ]:
tests

,name,t_statistics,p_value
0,gender,-20.510603,4.717427e-80
1,age,64.790149,0.000000e+00
2,tb,7.804222,1.326160e-14
3,db,1.694649,9.040943e-02
4,tp,28.752217,8.388832e-138
5,albumin,10.501527,1.038662e-24
6,A/G,9.077904,4.612309e-19
7,SGPT,106.699345,0.000000e+00
8,SGOT,48.950244,7.698876e-285
9,Alkphos,-14.809349,1.364798e-45


In [ ]:
#create a subdataframe with the characteristics with smallest pvalue score or the biggest t_statisctic score
df_after_ttest = df[['age','SGPT','SGOT','tp','Alkphos','healthy']]

In [ ]:
df_after_ttest

,age,SGPT,SGOT,tp,Alkphos,healthy
0,65,6.8,3.3,187,0.90,1
1,62,7.5,3.2,699,0.74,1
2,62,7.0,3.3,490,0.89,1
3,58,6.8,3.4,182,1.00,1
4,72,7.3,2.4,195,0.40,1
...,...,...,...,...,...,...
578,60,5.9,1.6,500,0.37,2
579,40,6.0,3.2,98,1.10,1
580,52,6.4,3.2,245,1.00,1
581,31,6.8,3.4,184,1.00,1


SVM

In [ ]:
X = df.values[:, :-1]
Y = df.values[:, -1]

In [ ]:
#better results normalizing the data by removing the mean and scaling to unit variance
from sklearn import preprocessing 
scaler = preprocessing.StandardScaler().fit(X)
X = scaler.transform(X)

In [ ]:
from sklearn.model_selection import KFold
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics

# Split the data to training and testing data 5-Fold
kfold = KFold(n_splits=5, shuffle=True)
#The dataframe that my metric will be saved
df_metrics= pd.DataFrame(columns=['Fold','Specificity','Sensitivity'])
#model
gnb = GaussianNB()

for i, (train, test) in enumerate(kfold.split(X)):
  gnb = GaussianNB()
  # fit the model
  gnb.fit(X[train], Y[train])
  #predict
  y_pred_NB = gnb.predict(X[test])
  #colnfusion matrix
  cm = metrics.confusion_matrix(Y[test],y_pred_NB)
  #calculate Accuracy,Specificity,Sensitivity
  total=sum(sum(cm))
  Specificity = cm[0,0]/(cm[0,0]+cm[0,1])
  Sensitivity = cm[1,1]/(cm[1,0]+cm[1,1])
  df_metrics.loc[len(df_metrics.index)] = [i,Specificity,Sensitivity] 

GeometricMean = sqrt (df_metrics['Specificity'].mean()*df_metrics['Sensitivity'].mean())
print("GeometricMean after students t test:",GeometricMean)


GeometricMean after students t test: 0.6203129387444447


In [ ]:
df_metrics

,Fold,Specificity,Sensitivity
0,0.0,0.407407,0.972222
1,1.0,0.388889,0.962963
2,2.0,0.371795,0.948718
3,3.0,0.387500,0.916667
4,4.0,0.448276,1.000000
